# Homework 10: Cleaning data with Regular Expressions

Time to use regular expressions!

# Hints and notes

### Opening files in subdirectories

Notice that this notebook might be **homework/**, but!!! the csvs and text files might be in **homework/scraped/** or **/homework/scraped/minutes_pdfs** or **/homework/pdfs/**. To open a file in a subdirectory, instead of having the filename be `"file.csv"` you'll just use `"some/subfolder/file.csv"`

### Opening text files

This will open up a file, read it in and show you the first 500 characters.

```python
contents = open("your-filename.txt").read()
contents[0:500]
```

> You might need `open("your-filename.txt", encoding="utf8").read()`

### Using regex

For some dumb reason you need to put `r` in front of the string you use when you're talking about regex. Just plain `"(\d\d\d)"` will usually work, but *sometimes* it won't and you'll need `r"(\d\d\d)`. It's best to just use the `r` all of the time, if you can remember!

### Using `.str.extract`

When you use `.str.extract`, you're always going to **capture one thing** and save it to a new column. You need to wrap the things you're interested in with parenthesis `(` `)`.

```python
df['phone_number'] = df['old_column'].str.extract(r"My phone number is (\d\d\d-\d\d\d-\d\d\d\d)")
```

### Setting pandas options

Pandas has a lot of options, like how many columns or rows it will show you, or how many characters it will show in a column before it stops showing you anything. Here are a few useful ones:

* `display.max_cols`: Number of columns to show at once
* `display.max_rows`: Number of rows to show at once
* `display.max_colwidth`: Maximum number of characters displayed from a string

You can set them using `pd.set_option("display.max_rows", 1000)`, for example, to show 1000 rows at a time. You can find a lot more at https://pandas.pydata.org/pandas-docs/stable/generated/pandas.set_option.html

### Regular expressions reference

I personally think http://www.regular-expressions.info/ is a wonderful wonderful reference (and tutorial), even if it's ugly! But here's a quick reference for you:

* `\d` is a digit
* `\d*` is zero or more digits 
* `\d+` is one or more digits
* `.` matches anything for ONE character
* `.*` is "give me anything forever"
* `\s` is whitespace, a.k.a. spaces and tabs
* `\w` is a word character, which includes capital and lowercase letters, numbers and hyphens.
* You can put `*` after anything, so `\w*` would mean "as many word characters as you can find"
* `\b` is a word boundary (you'll need the `r""` thing for this one)
* `( )` is a "capture group" for saving something
* `\1` is used when doing find/replace to say "put the first captured group here" (note, it's a dollar sign instead of a backslash in some editors)
* `[ABCDE]` is a character class, which means "match one of these, I don't care which"
* dollar sign means "end of the line"
* caret ^ means "beginning of the line"
* `\.` means "no really seriously I mean a period not just anything"
* You can use `\` with anything else that would normally be a special character, too, not just periods. `(` or `[` or whatever.

### Cleaning up extracted columns

Sometimes you get `\n` (newlines) or spaces or `\t` (tabs) or stuff at the beginning or the end of your column. `.str.strip()` will usually take care of that, just attach it after your `.str.extract()`

After you extract something, it's still a string even though you look at it and know it's a number. Use `.astype(int)` to turn it into an integer (no decimal) or `.astype(float)` to turn it into a float (yes decimal)

### Writing regular expressions in general

Even if I'm using regex in pandas or Python, I like to test them in my text editor with "Find." The highlighting really helps me see if I'm matching things! I also like to think "what stays the same?" when designing patterns, write those parts first, then fill in the blanks with what I want to capture.

## Importing

There might be more, I just wanted to put this up here for the `pd.set_option` part. It allows you to see a lot of content in a single column of pandas, which will be important for some parts below.

In [4]:
import re
import pandas as pd
pd.set_option('display.max_colwidth', 500)

# Part 1: Using `.str.extract` to pull data from columns in pandas

## 1.1 H&M

Open up `hm.csv` from the `scraped` directory. I want **four new columns**:

1. `price_original`, the original price, one of the new price
2. `price_discounted`, the discounted price
3. `pct_discount`, the percent discount
4. `article_id`, the article id (from the url)

Save as **hm_cleaned.csv**.

**Note:** When you look at it, it... won't look right. I don't know why, pandas is weird. Look at the `price` column by itself using `df['price']` before you write your regex.

**Tip:** Remember that `$` is a special regex symbol! You might need to escape it.

**Tip:** When doing `.str.extract`, the whole match doesn't get captured, only what you put `()` around! Think about anchoring to different points of the string, or things in the string.

**Tip:** Not all prices have cents!

**Tip:** Your first instinct about how to compute the percent discount is probably wrong

In [5]:
df_hm = pd.read_csv('10-homework/scraped/hm.csv')
df_hm.head()

,name,price,url
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A


In [6]:
# article number
df_hm['article_id'] = df_hm['url'].str.extract(r'product/(\d+)?')
df_hm.head()

,name,price,url,article_id
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N,13472
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D,35079
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A,72462
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C,58926
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A,74242


In [7]:
df_hm['price_original'] = df_hm['price'].str.extract(r'\$([0-9.]+)$').astype(
  float)
df_hm['price_discounted'] = df_hm['price'].str.extract(r'^\$([0-9.]+) '
                                                       r'\$').astype(float)
df_hm.head()

,name,price,url,article_id,price_original,price_discounted
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N,13472,129.00,59.99
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D,35079,17.99,6.99
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A,72462,17.99,7.99
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C,58926,12.99,6.99
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A,74242,24.99,17.99


In [8]:
df_hm['discount'] = df_hm.apply(lambda row: (row['price_original'] - row[
  'price_discounted']) / row['price_original'], axis=1)

# not *really* needed, since we already got the normalised value, and we 
# should always prefer the normalised value, but if you insist ...
df_hm['pct_discount'] = df_hm.apply(lambda row: row['discount'] * 100, axis=1)
df_hm.head()

,name,price,url,article_id,price_original,price_discounted,discount,pct_discount
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N,13472,129.00,59.99,0.534961,53.496124
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D,35079,17.99,6.99,0.611451,61.145081
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A,72462,17.99,7.99,0.555864,55.586437
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C,58926,12.99,6.99,0.461894,46.189376
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A,74242,24.99,17.99,0.280112,28.011204


In [9]:
df_hm.to_csv('10-homework/hm_cleaned.csv', index=False)

## 1.2 Sci-Fi Authors

Open up `sci-fi.csv` to clean. Get rid of the `\n` on the title and and give me six new columns:

* `avg_rating`
* `rating_count`
* `total_score`
* `score_votes`
* `series` the series the book belongs to
* `series_no` the book in the series that it is

For series, I'm talking about e.g. `(The Hunger Games, #1)` is `series` "The Hunter Games" and `series_no` 1.

Save as **sci-fi_cleaned.csv**.

**Tip:** You don't need regex to clean the title - there's a special thing that removes whitespace from the beginning/end of strings

**Tip:** Remember that `(` and `)` are special characters

**BONUS:** When you make the `total_score` column, pay close attention to it. If you notice the problem, fix it.

**BONUS:** You don't need these columns to be numbers, but life would be better if they were. 

In [10]:
df_scifi = pd.read_csv('10-homework/scraped/sci-fi.csv')
df_scifi.head()

,full_rating,full_score,rank,title,url
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,\nThe Handmaid's Tale\n,/book/show/38447.The_Handmaid_s_Tale
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"\nThe Hunger Games (The Hunger Games, #1)\n",/book/show/2767052-the-hunger-games
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"\nFrankenstein, or The Modern Prometheus\n",/book/show/18490.Frankenstein_or_The_Modern_Prometheus
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"\nA Wrinkle in Time (A Wrinkle in Time Quintet, #1)\n",/book/show/18131.A_Wrinkle_in_Time
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,\nThe Left Hand of Darkness\n,/book/show/18423.The_Left_Hand_of_Darkness


In [11]:
df_scifi['title'] = df_scifi['title'].str.strip()
df_scifi['full_score'] = df_scifi['full_score'].str.strip()
df_scifi.head()

,full_rating,full_score,rank,title,url
0,"4.07 avg rating — 785,502 ratings","score: 28,539,\n and\n292 people voted",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale
1,"4.34 avg rating — 5,212,935 ratings","score: 27,566,\n and\n282 people voted",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games
2,"3.76 avg rating — 922,308 ratings","score: 20,049,\n and\n205 people voted",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Prometheus
3,"4.04 avg rating — 702,272 ratings","score: 17,684,\n and\n185 people voted",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time
4,"4.06 avg rating — 77,664 ratings","score: 16,070,\n and\n165 people voted",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness


In [12]:
# dealing with the titles and the series
df_scifi['series'] = df_scifi['title'].str.extract(r'\((.+), #')
df_scifi['series_no'] = df_scifi['title'].str.extract(r'#(\d+)\)')
df_scifi.tail()

,full_rating,full_score,rank,title,url,series,series_no
95,"4.17 avg rating — 3,717 ratings","score: 1,197,\n and\n16 people voted",96,"Invader (Foreigner, #2)",/book/show/57014.Invader,Foreigner,2
96,"3.78 avg rating — 103,428 ratings","score: 1,174,\n and\n13 people voted",97,"Across the Universe (Across the Universe, #1)",/book/show/8235178-across-the-universe,Across the Universe,1
97,4.16 avg rating — 97 ratings,"score: 1,170,\n and\n13 people voted",98,"A Lost Kitten (Sea-anan Saga, #1)",/book/show/16029924-a-lost-kitten,Sea-anan Saga,1
98,3.85 avg rating — 111 ratings,"score: 1,169,\n and\n13 people voted",99,"Love Between the Hired Hands (The Men of Space Station One, #4)",/book/show/13482469-love-between-the-hired-hands,The Men of Space Station One,4
99,"3.83 avg rating — 3,244 ratings","score: 1,155,\n and\n16 people voted",100,Ammonite,/book/show/180270.Ammonite,NaN,NaN


In [13]:
# Rating stuff

# okay, we have some ugly formatted numbers, import the locale library and 
# set the potentially correct locale
import locale

locale.setlocale(locale.LC_NUMERIC, 'en_US.UTF-8')


def to_int(num):
  return locale.atoi(num)


df_scifi['avg_rating'] = df_scifi['full_rating'] \
  .str.extract(r'([0-9.]+) avg rating').astype(float)
df_scifi['rating_count'] = df_scifi['full_rating'] \
  .str.extract(r'([0-9,]+) ratings$')
df_scifi['rating_count'] = df_scifi['rating_count'].apply(to_int)
df_scifi.head()


,full_rating,full_score,rank,title,url,series,series_no,avg_rating,rating_count
0,"4.07 avg rating — 785,502 ratings","score: 28,539,\n and\n292 people voted",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale,NaN,NaN,4.07,785502
1,"4.34 avg rating — 5,212,935 ratings","score: 27,566,\n and\n282 people voted",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games,The Hunger Games,1,4.34,5212935
2,"3.76 avg rating — 922,308 ratings","score: 20,049,\n and\n205 people voted",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Prometheus,NaN,NaN,3.76,922308
3,"4.04 avg rating — 702,272 ratings","score: 17,684,\n and\n185 people voted",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time,A Wrinkle in Time Quintet,1,4.04,702272
4,"4.06 avg rating — 77,664 ratings","score: 16,070,\n and\n165 people voted",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness,NaN,NaN,4.06,77664


In [14]:
# Score stuff

df_scifi['total_score'] = df_scifi['full_score']\
  .str.extract(r'score: ([0-9,]+),')
df_scifi['score_votes'] = df_scifi['full_score']\
  .str.extract(r'([0-9,]+) people voted')

df_scifi['total_score'] = df_scifi['total_score'].apply(to_int)
df_scifi['score_votes'] = df_scifi['score_votes'].apply(to_int)

df_scifi.tail()

,full_rating,full_score,rank,title,url,series,series_no,avg_rating,rating_count,total_score,score_votes
95,"4.17 avg rating — 3,717 ratings","score: 1,197,\n and\n16 people voted",96,"Invader (Foreigner, #2)",/book/show/57014.Invader,Foreigner,2,4.17,3717,1197,16
96,"3.78 avg rating — 103,428 ratings","score: 1,174,\n and\n13 people voted",97,"Across the Universe (Across the Universe, #1)",/book/show/8235178-across-the-universe,Across the Universe,1,3.78,103428,1174,13
97,4.16 avg rating — 97 ratings,"score: 1,170,\n and\n13 people voted",98,"A Lost Kitten (Sea-anan Saga, #1)",/book/show/16029924-a-lost-kitten,Sea-anan Saga,1,4.16,97,1170,13
98,3.85 avg rating — 111 ratings,"score: 1,169,\n and\n13 people voted",99,"Love Between the Hired Hands (The Men of Space Station One, #4)",/book/show/13482469-love-between-the-hired-hands,The Men of Space Station One,4,3.85,111,1169,13
99,"3.83 avg rating — 3,244 ratings","score: 1,155,\n and\n16 people voted",100,Ammonite,/book/show/180270.Ammonite,NaN,NaN,3.83,3244,1155,16


## 1.3 Where you're just doing one of my former students' projects

Once upon a time my student Stefan did a project that involved some lawyer stuff. Most of the content was in PDFs, though! I converted them to text files and put them into the `pdfs` folder, and gave you code below to open up each of them and save their contents into a dataframe.

What a nice dataframe! I want you to add the following columns to it:

* `lawyer_app`, the applicant's lawyer (pro se means that they did it themselves, that's fine)
* `lawyer_gov`, the government's lawyer
* `judge`, the name of the judge
* `access`, whether the clearance is granted or denied (although you might miss a few)

Save as **court_cleaned.csv**.

**Note:** You can look at the original PDFs, they're also included.

**Note:** This uses a fun utility called `glob`, which is mostly fun because you use it as `glob.glob`. It's used to find files that match a certain filename pattern.

**BONUS:** You'll be happy once you get the judge, but make sure it doesn't have any extra punctuation on it.

**BONUS:** You can for some words using `.str.contains("blah")` and save it into new columns. Maybe `has_debt`, `has_bankruptcy`, etc.

> It's okay if it isn't perfect. Converting PDF into data rarely is! Usually you get 90% of it done with computers, then send people to enter the other 10% by hand.

In [15]:
import glob

filenames = glob.glob("10-homework/pdfs/*.txt")
contents = [open(filename, encoding="utf8").read() for filename in filenames]
df = pd.DataFrame({'filename': filenames, 'content': contents})
df.head()

,filename,content
0,10-homework/pdfs/11-11916.h1.pdf.txt,"\n\nDEPARTMENT OF DEFENSE \n\nDEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\nREDACTED \n\n \n\nISCR Case No. 11-11916 \n\nMENDEZ, Francisco, Administrative Judge: \n\n \nApplicant did not mitigate security concerns raised by his exercise of foreign \ncitizenship, including the possession of a current foreign passpor..."
1,10-homework/pdfs/12-01601.a1.pdf.txt,"KEYWORD: Guideline F\n\nDIGEST: Most of the document Applicant submitted on appeal were not previously submitted to\nthe Judge and the Board is prohibited from receiving or considering them. Adverse decision\naffirmed.\n\nCASENO: 12-01601.a1\n\nDATE: 09/23/2016\n\nIn Re:\n\n-------\n\nApplicant for Security Clearance\n\nDATE: September 23, 2016\n\nISCR Case No. 12-01601\n\n)\n)\n)\n)\n)\n)\n)\n)\n\nAPPEAL BOARD DECISION\n\nAPPEARANCES\n\nFOR GOVERNMENT\n\nJames B. Norman, Esq., Chief Depart..."
2,10-homework/pdfs/11-03073.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n\n \n\nISCR Case No. 11-03073 \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\nFor Government: Robert J. Kilmartin, Esq., Department Counse..."
3,10-homework/pdfs/11-04909.h1.pdf.txt,"\n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n DEPARTMENT OF DEFENSE \n\n \n \n\n \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-04909 \n\n \n\nFor Government: Richard Stevens, Esq., Department Counsel \n\nFor Applicant: Pro se \n\n \n\n \nDUFFY, James F., Administrative Judge: \n\n \nApplicant mitigated \n\nconsiderations). Clearance is ..."
4,10-homework/pdfs/11-08313.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n\nISCR Case No. 11-08313 \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n--------------- \n \n\n \n\n \n \n\n) \n) \n) \n) \n) \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\nFor Government: Julie R. Mendez, Esquire, Department Counsel \n\nFo..."


In [16]:
# * `lawyer_app`, the applicant's lawyer (pro se means that they did it themselves, that's fine)

df['lawyer_app'] = df['content']\
  .str.extract(r'''
  for\ applicant # anchor
  :? # there is not always a colon
  (?:\w|\n*) # white space or a new line
  (.+)\n # get everything until you reach the new line
  ''', re.IGNORECASE+re.VERBOSE)
df['lawyer_app'] = df['lawyer_app'].str.strip()

# * `lawyer_gov`, the government's lawyer

df['lawyer_gov'] = df['content'] \
  .str.extract(r'''
  for\ government # anchor
  :? # there is not always a colon
  (?:\w|\n*) # white space or a new line
  (.+)\n # get everything until you reach the new line
  ''', re.IGNORECASE + re.VERBOSE)
df['lawyer_gov'] = df['lawyer_gov'].str.strip()

# * `judge`, the name of the judge

df['judge'] = df['content'].str.extract(r'''
^(.+)
,\ administrative\ judge
''', re.IGNORECASE + re.VERBOSE + re.MULTILINE)

# * `access`, whether the clearance is granted or denied (although you might miss a few)

df['access'] = df['content'].str.contains(r'is granted')

df.count()

filename      15
content       15
lawyer_app    15
lawyer_gov    15
judge         13
access        15
dtype: int64

Okay, now do the work and **make those new columns!**

# Reading books

When you're doing text work, you're legally obligated work on Jane Austen's Pride and Prejudice (at least I *think* so). Let's do some naive analysis of it!

## Read in Jane Austen's Pride and Prejudice (without moving the file!)

It's in the `data/` directory, and named `Austen_Pride.txt`.

In [17]:
austen = open('10-homework/data/Austen_Pride.txt').read()


## Look at the first 500 or so characters of it 

In [18]:
austen[:500]

' Pride and Prejudice\nby Jane Austen\nChapter 1\nIt is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.\nHowever little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.\n"My dear Mr. Bennet," said his lady to him one day, "have you heard that Nethe'

## Use a regular expression to find every "he" or "she" in the book. There should be about 3000 of them.

**Tip:** Do you know about **word boundaries?** `\b` means "the beginning of end of a word."

**Tip:** You might also want to use `re.IGNORECASE`. Maybe you'll need to google it? 

**Tip:** Do NOT use `re.compile`

In [26]:
matches = re.findall(r'\bs?he\b', austen, re.IGNORECASE)


3047

## Use a regular expression to find those same "he" or "she"s, but also match *the word after it*

The first four should be:

* he is
* he had
* she told
* he came

In [29]:
activities = re.findall(r'\bs?he \w+', austen, re.IGNORECASE)
activities[:10]

['he is',
 'he had',
 'she told',
 'he came',
 'he agreed',
 'he is',
 'he married',
 'he may',
 'he comes',
 'she ought']

## Use capture groups to save the pronoun (he/she) as one match and the word as another

The first five should look like

```
[('he', 'is'),
 ('he', 'had'),
 ('she', 'told'),
 ('he', 'came'),
 ('he', 'agreed')]```

In [30]:
split_activities = re.findall(r'\b(s?he) (\w+)', austen, re.IGNORECASE)
split_activities[:10]

[('he', 'is'),
 ('he', 'had'),
 ('she', 'told'),
 ('he', 'came'),
 ('he', 'agreed'),
 ('he', 'is'),
 ('he', 'married'),
 ('he', 'may'),
 ('he', 'comes'),
 ('she', 'ought')]

## Save those matches into a dataframe

You can give the column names with `columns=['pronoun', 'verb']`

In [31]:
df_austen = pd.DataFrame(split_activities, columns=['pronoun', 'verb'])
df_austen.head()

,pronoun,verb
0,he,is
1,he,had
2,she,told
3,he,came
4,he,agreed


## How many times is each pronoun used?

In [32]:
df_austen['pronoun'].value_counts()

she    1322
he     1054
She     325
He      234
Name: pronoun, dtype: int64

## Oh, wait, clean that up.

Make it only 'he' and 'she' lowercase.

It should be about 1600 'she' and 1300 'he'

In [35]:
df_austen['pronoun'] = df_austen['pronoun'].str.lower()
df_austen['pronoun'].value_counts()

she    1647
he     1288
Name: pronoun, dtype: int64

## What are the top 20 most common verbs?

In [37]:
df_austen['verb'].value_counts().head(20)

was          372
had          371
could        172
is           139
would         94
has           70
did           67
will          50
might         46
should        41
felt          38
must          37
said          33
saw           32
added         31
thought       31
then          26
replied       22
continued     21
looked        21
Name: verb, dtype: int64

## What are the top 20 most common verbs for 'he', and the top 20 most common for 'she'

**Tip:** Don't use groupby, just filter. If you want to know how, though, you can also look at "value counts for different categories" on [this page](http://jonathansoma.com/lede/foundations-2017/classes/more-pandas/class-notes/)

In [39]:
df_austen[df_austen['pronoun'] == 'he']['verb'].value_counts().head(20)

had        166
was        160
is          74
has         49
could       40
would       35
did         29
should      26
must        24
will        24
might       21
replied     14
came        13
said        13
thought     13
may         13
does        12
never       12
meant       11
then        11
Name: verb, dtype: int64

In [41]:
df_austen[df_austen['pronoun'] == 'she']['verb'].value_counts().head(20)

was        212
had        205
could      132
is          65
would       59
did         38
felt        33
saw         29
will        26
might       25
added       23
has         21
said        20
thought     18
should      15
then        15
found       13
must        13
soon        12
wished      11
Name: verb, dtype: int64

## Who cries more, men or women? Give me a percentage answer.

**Tip:** It's `cried`, because of, you know, how books are written

In [42]:
df_austen[df_austen['verb'] == 'cried']['pronoun'].value_counts(normalize=True)

she    0.916667
he     0.083333
Name: pronoun, dtype: float64

## How much more common is 'he' than 'she' in J.R.R. Tolkein's Fellowship of the Ring? How does that compare to Pride and Prejudice?

The book is in the same directory.

In [43]:
fs_text = open('10-homework/data/Lord of the Rings - 01 - The Fellowship of '
              'the Ring - J. R. R. Tolkien - 1955.txt').read()
fs_pronouns = re.findall(r'\bs?he\b', fs_text, re.IGNORECASE)
fs_df = pd.DataFrame(fs_pronouns, columns=['pronoun'])
fs_df['pronoun'] = fs_df['pronoun'].str.lower()
fs_df['pronoun'].value_counts(normalize=True)

he     0.950636
she    0.049364
Name: pronoun, dtype: float64

In [61]:
print('"He" is in The Fellowship of the Ring {:.1f} times more common that '
      '"she"'.format(
  fs_df['pronoun'].value_counts()[0] / fs_df['pronoun'].value_counts()[1]))


"He" is in The Fellowship of the Ring 19.3 times more common that "she"


In [45]:
df_austen['pronoun'].value_counts(normalize=True)

she    0.561158
he     0.438842
Name: pronoun, dtype: float64

In [63]:
print('"She" is in Pride and Prejudice {:.1f} times more common that '
      '"he"'.format(
  df_austen['pronoun'].value_counts()[1] / df_austen['pronoun'].value_counts(
    
  )[0]))


"She" is in Pride and Prejudice 0.8 times more common that "he"
